### Importing Libraries


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math


### Initialization of parameters

In [ ]:
mu_r = 2.0
epsilon_r = 6.0
delta_z = 0.004293 #converted from cm to m
N_z = 400
n_r = math.sqrt((mu_r*epsilon_r))
z = np.zeros((N_z,1))
#initialization of device parameters
Mu_r = np.ones((1,N_z))
Epsilon_r = np.ones((1,N_z))
print(Mu_r,Epsilon_r)
print(Mu_r.size,Epsilon_r.size)
#Adding the materials on the grid

Mu_r[:,13:83] = mu_r
Epsilon_r[:,13:83] = epsilon_r
print(Mu_r.shape,Epsilon_r.shape,z.shape)

### Initial computations of relevant parameters

In [ ]:
#Compute time step
c_0 =  299792458 #m/s
delta_t = (1*delta_z)/(2*c_0)
print(delta_t)

#Compute Source parameters
tau = 1/(2*1E9) #Width of gaussian pulse
t_0 = 6*tau     #offset in time for source
print(tau,t_0)

#Compute number of iterations
t_prop = (n_r*N_z*delta_z)/c_0
T = 12*tau + 5*t_prop
iteration = math.ceil(T/delta_t)
t = np.linspace(0,iteration*delta_t,iteration)
print(T,iteration,t.shape)

### Defining source excitation

In [ ]:
def gaussian_source(t,t_0,tau,delta_t,delta_z,c_0):
    #Set source permittivity and permeability
    mu_src =1 #these parameters should be the material permittivity/permeability at the grid position of the source injection
    epsilon_src = 1
    n_src = np.sqrt(epsilon_src*mu_src)
    delta = (n_src*delta_z/2*c_0)+(delta_t/2)
    A = -np.sqrt(epsilon_src/mu_src)
    x = (t - t_0)/tau
    Esrc = np.exp(-np.power(x,2))
    Hsrc = A*np.exp(-np.power(x,2))
    return Esrc,Hsrc

### Source excitation initialization

In [ ]:
Esrc,Hsrc = gaussian_source(t,t_0,tau,delta_t,delta_z,c_0)
print(z.shape,Esrc.shape,Hsrc.shape)
plt.figure(figsize=[20,5])
plt.plot(t,Esrc)
plt.plot(t,Hsrc)

### Main FDTD Loop (Basic Algorithm)

In this part, there is no excitation so n

In [ ]:
#FDTD Loop
#Creating Field values
E = np.zeros((1,N_z))
H_norm = np.zeros((1,N_z))
#Update coefficient values
m_E = c_0*delta_t/1
m_H = c_0*delta_t/1
E.shape,H_norm.shape
plt.figure(1)
#Update Equations (Looping in time)
for i in range(iteration):
    #Looping in space
    for j in range(N_z-1):
        H_norm[:,j] = H_norm[:,j] + m_H*((E[:,j+1] - E[:,j])/delta_z)
    #Boundary condition
    H_norm[:,N_z-1] = H_norm[:,N_z-1] + m_H*(-E[:,N_z-1]/delta_z)
    
    #Boundary condition
    E[:,0] = E[:,j] + m_E*((H_norm[:,j]-H_norm[:,j-1])/delta_z)
    #looping in space (Electric Field)
    for j in range(1,N_z):
        E[:,j] = E[:,j] + m_E*(H_norm[:,j]/delta_z)
    
    
    
    plt.plot(E.T)#Plot E-Field
    plt.plot(H_norm.T)#Plot H-Field
    plt.ylabel('Value')
    plt.xlabel('z (Space)')
    plt.title(f'FDTD 1-D Iteration step: {i}/{iteration}')
    plt.show()
    #lineE.set_xdata(z)
    #lineE[0].set_ydata(E[i].T)
    #lineH.set_xdata(z)
    ##lineH[0].set_ydata(H_norm[i].T)
    #plt.pause(1)
    #fig.show()
    
print(E.shape,H_norm.shape)
        

#### FDTD Loop (Algorithm with Soft source)

In [ ]:
#FDTD Loop
#Creating Field values
E = np.zeros((1,N_z))
H_norm = np.zeros((1,N_z))
#Update coefficient values
m_E = c_0*delta_t/1
m_H = c_0*delta_t/1
E.shape,H_norm.shape
#Update Equations (Looping in time)
for i in range(iteration):
    #Looping in space
    for j in range(N_z-1):
        H_norm[:,j] = H_norm[:,j] + m_H*((E[:,j+1] - E[:,j])/delta_z)
        if j == 3: #Injecting the source
            H_norm[:,j] = H_norm[:,j] + Hsrc[i]
    #Boundary condition
    H_norm[:,N_z-1] = H_norm[:,N_z-1] + m_H*(-E[:,N_z-1]/delta_z)
    
    #Boundary condition
    E[:,0] = E[:,j] + m_E*(H_norm[:,0]/delta_z)
    #looping in space (Electric Field)
    for j in range(1,N_z):
        E[:,j] = E[:,j] + m_E*((H_norm[:,j] - H_norm[:,j-1])/delta_z)
        if j == 3: #Injecting the source
            E[:,j] = E[:,j] + Esrc[i]
 
    plt.figure(1,[30,10])
    plt.plot(E.T)#Plot E-Field
    plt.plot(H_norm.T)#Plot H-Field
    plt.ylabel('Value')
    plt.xlabel('z (Space)')
    plt.title(f'FDTD 1-D Iteration step: {i}/{iteration}')
    plt.show()
    #lineE.set_xdata(z)
    #lineE[0].set_ydata(E[i].T)
    #lineH.set_xdata(z)
    ##lineH[0].set_ydata(H_norm[i].T)
    #plt.pause(1)
    #fig.show()
    
print(E.shape,H_norm.shape)

In [ ]:
import matplotlib.animation as animation

def animate(i):
    line1.set_ydata(E[i,:])  # update the data.
    line2.set_ydata(H_norm[i,:])
    return line1,line2,

fig, ax = plt.subplots()
line1, = ax.plot(z,E[0])#Plot E-Field
line2, = ax.plot(z,H_norm[0])#Plot H-Field

ani = animation.FuncAnimation(
    fig, animate, interval=iteration, blit=True, save_count=50)

# To save the animation, use e.g.
#
# ani.save("movie.mp4")
#
# or
#
# writer = animation.FFMpegWriter(
#     fps=15, metadata=dict(artist='Me'), bitrate=1800)
# ani.save("movie.mp4", writer=writer)

plt.show()

In [ ]:
E.shape,H_norm.shape

In [ ]:
number steps in time x number grids